In [8]:
import os, pandas as pd

csv_url = os.path.join(os.getcwd(), "../data/whiskies.csv")

In [9]:
csv_url

'/Users/mainframe/Workspace/ML/WV-Team/notebooks/../data/whiskies.csv'

In [10]:
# read the CSV file located in data/whiskies.csv
df = pd.read_csv(csv_url)

# show the first few rows
df.head()

,id,name,name_eng,category,brand,nation,external_image_url,image_url,full_image_url,region,distillery
0,60600124,1800 레포사도,1800 Reposado,기타,1800,멕시코,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/6060...,NaN,Fábrica La Rojeña
1,60600125,1800 레포사도 38%,1800 Reposado 38%,기타,1800,멕시코,https://d1e2y5wc27crnp.cloudfront.net/media/sm...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/6060...,NaN,Fábrica La Rojeña
2,60600126,1800 실버,1800 Silver,기타,1800,멕시코,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/6060...,NaN,Fábrica La Rojeña
3,64000050,플로르 데 카냐 25년,Flor de Caña 25yo,기타,Flor de Caña,니카라과,https://d1e2y5wc27crnp.cloudfront.net/media/co...,NaN,NaN,NaN,Flor de Caña
4,10101959,보모어 50년,Bowmore 50yo,싱글몰트,Bowmore,스코틀랜드,https://d1e2y5wc27crnp.cloudfront.net/media/co...,https://cdn.whisket.co.kr/public/whiskies_dsh/...,https://cdn.whisket.co.kr/public/whiskies/1010...,아일라 섬,Bowmore


In [ ]:
import requests
from urllib.parse import urlparse

save_dir = os.path.join(os.getcwd(), "../data/photos")
os.makedirs(save_dir, exist_ok=True)

def get_extension_from_url(url):
    """Extract extension from URL if possible, else use .jpg"""
    path = urlparse(url).path
    ext = os.path.splitext(path)[1]
    if ext.lower() in [".jpg", ".jpeg", ".png", ".webp"]:
        return ext
    return ".jpg"

def download_image(url, dest_path):
    """Download a single image safely"""
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        with open(dest_path, "wb") as f:
            f.write(resp.content)
        print(f"✅ Saved {dest_path}")
    except Exception as e:
        print(f"❌ Failed {url}: {e}")

# Iterate through each row
for _, row in df.iterrows():
    whisky_id = row["id"]

    urls = [row.get("external_image_url"), row.get("full_image_url")]
    for idx, url in enumerate(urls, start=1):
        if pd.isna(url) or not isinstance(url, str):
            continue

        ext = get_extension_from_url(url)
        dest_path = os.path.join(save_dir, f"{whisky_id}_{idx}{ext}")

        # Skip if already downloaded
        if os.path.exists(dest_path):
            continue

        download_image(url, dest_path)

In [12]:
# Drop rows with NaN in full_image_url
filtered = df.dropna(subset=["full_image_url"])

# Drop rows where external_image_url starts with "/m/"
filtered = filtered[~filtered["external_image_url"].astype(str).str.startswith("/m/")]

print(f"Before: {len(df)} rows")
print(f"After : {len(filtered)} rows")

Before: 2958 rows
After : 2145 rows


In [14]:
#Save the data just filtered
filtered_csv_url = os.path.join(os.getcwd(), "../data/whiskies_filtered.csv")

filtered.to_csv(filtered_csv_url, index=False)